In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading csv files
games_women = pd.read_csv('athletes-women.csv')
games_men = pd.read_csv('athletes-men.csv')
athletes = pd.read_csv('athletes-all.csv')

# athletes who made it into the games
games_all = pd.concat([games_women, games_men])
games_women = games_women.drop(columns = ['rank', 'latitude', 'longitude', 'name', 'region'])
games_men = games_men.drop(columns = ['rank', 'latitude', 'longitude', 'name', 'region'])

/Users/nadinelin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = athletes.merge(games_all.drop_duplicates(), on=['athlete_id'],
                   how='left', indicator=True)

In [4]:
# taking only the columns we want
df_all = df[['athlete_id', 'age_x', 'gender', 'height_x', 'weight_x', 'fran_x', 'run5k_x', 'pullups_x', '_merge', 'train', 'experience', 'schedule', 'howlong_x', 'run400_x', 'candj_x', 'snatch_x', 'deadlift_x', 'backsq_x']]
df_all = df_all.dropna()

df_all_women = df_all.loc[df_all['gender'] == 'Female']

In [5]:
# ones who have not made it who have stats filled out, temp while cleaning up
def make_columns_numeric(df):    
    not_make = df.loc[df['_merge'] == 'left_only']
    not_make['howlong_x'] = not_make['howlong_x'].str.slice(0, 6)

    col         = 'howlong_x'
    conditions  = [ not_make[col].str.contains("Less"), not_make[col].str.contains("6-12"), not_make[col].str.contains("1-2"), not_make[col].str.contains("2-4"), not_make[col].str.contains("4+") ]
    choices     = [0,1,2,3,4]

    not_make["howlong"] = np.select(conditions, choices)

    not_make['coach'] = np.where(not_make['train'].str.contains("have a coach"), 1, 0)
    not_make['certification'] = np.where(not_make['experience'].str.contains("Level 1"), 1, 0)
    not_make['multiple'] = np.where(not_make['schedule'].str.contains("multiple workouts"), 1, 0)

    return not_make

not_make_women = make_columns_numeric(df_all_women)

<ipython-input-5-1facb267e819>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_make['howlong_x'] = not_make['howlong_x'].str.slice(0, 6)
<ipython-input-5-1facb267e819>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_make["howlong"] = np.select(conditions, choices)
<ipython-input-5-1facb267e819>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [6]:
#not_in_games will contain all athletes that did not make it, with stats, with cleaned up columns
not_in_games_women = not_make_women[['athlete_id', 'age_x', 'height_x', 'weight_x', 'fran_x', 'run5k_x', 'pullups_x', 'run400_x', 'candj_x', 'snatch_x', 'deadlift_x', 'backsq_x','howlong','coach','certification','multiple']]
not_in_games_women.rename(columns={"age_x": "age",'height_x': 'height', 'weight_x':'weight', 'fran_x':'fran', 'run5k_x':'run5k','pullups_x':'pullups', 'run400_x':'run400', 'candj_x':'candj', 'snatch_x':'snatch', 'deadlift_x':'deadlift', 'backsq_x':'backsq'}, inplace = True)

/Users/nadinelin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [7]:
#Min and Max of women
#On August 31st, Marisa Flowers performed the CrossFit® WOD “Fran” in 1 minute 53 seconds.
not_in_games_women2 = not_in_games_women.loc[(not_in_games_women["height"] <= 108) & (not_in_games_women["weight"] <= 1400) &(not_in_games_women["fran"] <= 600) & (not_in_games_women["fran"] >= 113) & (not_in_games_women["run400"] <= 180)]
not_in_games_women2.agg([min, max])



,athlete_id,age,height,weight,fran,run5k,pullups,run400,candj,snatch,deadlift,backsq,howlong,coach,certification,multiple
min,1659.0,15.0,1.0,55.0,125.0,480.0,0.0,51.0,0.0,0.0,0.0,0.0,0,0,0,0
max,591544.0,55.0,74.0,235.0,600.0,2565.0,100.0,170.0,355.0,285.0,606.0,551.0,4,1,1,1


In [8]:
games_women['in_games'] = 1
not_in_games_women2['in_games'] = 0

<ipython-input-8-616d7dcee3e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_in_games_women2['in_games'] = 0


In [9]:
df_women = not_in_games_women2.append(games_women)

# Drop ID column
df_women = df_women.drop(['athlete_id'],axis=1)

In [10]:
df_women

,age,height,weight,fran,run5k,pullups,run400,candj,snatch,deadlift,backsq,howlong,coach,certification,multiple,in_games
64482,32.0,66.0,135.0,264.0,1355.0,36.0,75.0,165.0,125.0,266.0,195.0,3,1,0,0,0
97672,32.0,69.0,185.0,455.0,2172.0,20.0,137.0,115.0,85.0,205.0,155.0,2,0,0,0,0
144480,40.0,64.0,168.0,380.0,1753.0,6.0,108.0,140.0,90.0,295.0,225.0,2,1,0,1,0
159571,29.0,64.0,135.0,159.0,1195.0,44.0,65.0,210.0,163.0,330.0,270.0,4,1,1,1,0
159574,34.0,61.0,130.0,164.0,1269.0,37.0,70.0,173.0,138.0,333.0,253.0,4,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,53.0,62.0,130.0,396.0,1224.0,40.0,65.0,135.0,90.0,280.0,165.0,4,1,0,1,1
221,20.0,65.0,134.0,405.0,1630.0,28.0,79.0,139.0,143.0,254.0,209.0,2,0,0,0,1
222,22.0,65.0,147.0,554.0,1525.0,25.0,69.0,155.0,125.0,255.0,205.0,1,1,0,1,1
223,50.0,62.0,159.0,235.0,1353.0,51.0,71.0,155.0,125.0,270.0,225.0,3,0,0,1,1


In [11]:
pos_tempo = df_women[df_women['in_games']== 1]['age']
neg_tempo = df_women[df_women['in_games']== 0]['age']
pos_tempo = df_women[df_women['in_games']== 1]['height']
neg_tempo = df_women[df_women['in_games']== 0]['height']
pos_tempo = df_women[df_women['in_games']== 1]['weight']
neg_tempo = df_women[df_women['in_games']== 0]['weight']
pos_tempo = df_women[df_women['in_games']== 1]['fran']
neg_tempo = df_women[df_women['in_games']== 0]['fran']
pos_tempo = df_women[df_women['in_games']== 1]['run5k']
neg_tempo = df_women[df_women['in_games']== 0]['run5k']
pos_tempo = df_women[df_women['in_games']== 1]['pullups']
neg_tempo = df_women[df_women['in_games']== 0]['pullups']
pos_tempo = df_women[df_women['in_games']== 1]['run400']
neg_tempo = df_women[df_women['in_games']== 0]['run400']
pos_tempo = df_women[df_women['in_games']== 1]['candj']
neg_tempo = df_women[df_women['in_games']== 0]['candj']
pos_tempo = df_women[df_women['in_games']== 1]['snatch']
neg_tempo = df_women[df_women['in_games']== 0]['snatch']
pos_tempo = df_women[df_women['in_games']== 1]['deadlift']
neg_tempo = df_women[df_women['in_games']== 0]['deadlift']
pos_tempo = df_women[df_women['in_games']== 1]['backsq']
neg_tempo = df_women[df_women['in_games']== 0]['backsq']

In [12]:
train, test = train_test_split(df_women,test_size=0.15)

In [13]:
c = DecisionTreeClassifier(min_samples_split=100)

In [14]:
features = ['age', 'height', 'weight', 'fran', 'run5k', 'pullups', 'run400', 'candj', 'snatch', 'deadlift', 'backsq']

In [15]:
X_train = train[features]
y_train = train['in_games']

X_test = test[features]
y_test = test['in_games']

In [16]:
c.fit(X_train,y_train)

DecisionTreeClassifier(min_samples_split=100)

In [17]:
import imageio,io
import pydotplus

In [18]:
def show_tree(tree,features,path):
  f = io.StringIO()
  export_graphviz(tree,out_file=f,feature_names=features)
  pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
  img = imageio.imread(path)
  plt.rcParams['figure.figsize'] = (20,20)
  plt.imshow(img)

In [19]:
show_tree(c,features,'decision_tree')

InvocationException: GraphViz's executables not found

In [38]:
pip install graphviz

     |████████████████████████████████| 46 kB 5.7 MB/s eta 0:00:011
You should consider upgrading via the '/Users/nadinelin/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [40]:
y_pred = c.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)*100

In [42]:
print("Accuracy using Decision Tree: ",score )

Accuracy using Decision Tree:  90.22988505747126
